<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
# 1. Импорт библиотек
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import seaborn as sns

# 2. Загрузка переменных окружения
load_dotenv()

API_URL = os.getenv('API_URL')
DATE_BEGIN = os.getenv('DATE_BEGIN')
DATE_END = os.getenv('DATE_END')

# 3. Создание папки для графиков
os.makedirs('./charts', exist_ok=True)

# 4. Загрузка данных с API
visits = requests.get(f"{API_URL}/visits", params={'begin': DATE_BEGIN, 'end': DATE_END}).json()
registrations = requests.get(f"{API_URL}/registrations", params={'begin': DATE_BEGIN, 'end': DATE_END}).json()

# 5. Преобразование в датафреймы
visits_df = pd.DataFrame(visits)
registrations_df = pd.DataFrame(registrations)

# 6. Предобработка и фильтрация визитов
visits_df['datetime'] = pd.to_datetime(visits_df['datetime'])
visits_df['date_group'] = visits_df['datetime'].dt.date
visits_df = visits_df[~visits_df['user_agent'].str.contains('bot', case=False)]

# 7. Группировка визитов
visits_grouped = visits_df.groupby(['date_group', 'platform']).agg(visits=('visit_id', 'nunique')).reset_index()

# 8. Предобработка регистраций
registrations_df['datetime'] = pd.to_datetime(registrations_df['datetime'])
registrations_df['date_group'] = registrations_df['datetime'].dt.date

# 9. Группировка регистраций
registrations_grouped = registrations_df.groupby(['date_group', 'platform']).agg(registrations=('user_id', 'nunique')).reset_index()

# 10. Объединение и расчет конверсии
merged_df = pd.merge(visits_grouped, registrations_grouped, on=['date_group', 'platform'], how='outer').fillna(0)
merged_df['conversion'] = (merged_df['registrations'] / merged_df['visits'].replace({0: np.nan})) * 100
merged_df['conversion'] = merged_df['conversion'].fillna(0)

# 11. Сохранение в JSON
merged_df.to_json('./conversion.json')

# 12. Загрузка и обработка рекламы
ads_df = pd.read_csv('./ads.csv', parse_dates=['date'])
ads_df['date_group'] = ads_df['date'].dt.date
ads_grouped = ads_df.groupby('date_group').agg(
    cost=('cost', 'sum'),
    utm_campaign=('utm_campaign', lambda x: ', '.join(set(x)))
).reset_index()

# 13. Объединение с основной таблицей
ads_merged = pd.merge(
    merged_df.groupby('date_group').agg({
        'visits': 'sum',
        'registrations': 'sum'
    }).reset_index(),
    ads_grouped,
    on='date_group',
    how='left'
)

ads_merged['utm_campaign'] = ads_merged['utm_campaign'].fillna('none')
ads_merged['cost'] = ads_merged['cost'].fillna(0)

# 14. Сохранение в JSON
ads_merged.to_json('./ads.json')

# 15. Визуализация
sns.set(style="whitegrid")

# Итоговые визиты
plt.figure(figsize=(10, 5))
merged_df.groupby('date_group')['visits'].sum().plot()
plt.title('Итоговые визиты')
plt.xlabel('Дата')
plt.ylabel('Визиты')
plt.tight_layout()
plt.savefig('./charts/total_visits.png')
plt.close()

# Итоговые визиты по платформам
plt.figure(figsize=(10, 5))
sns.lineplot(data=merged_df, x='date_group', y='visits', hue='platform')
plt.title('Визиты по платформам')
plt.xlabel('Дата')
plt.ylabel('Визиты')
plt.tight_layout()
plt.savefig('./charts/visits_by_platform.png')
plt.close()

# Итоговые регистрации
plt.figure(figsize=(10, 5))
merged_df.groupby('date_group')['registrations'].sum().plot()
plt.title('Итоговые регистрации')
plt.xlabel('Дата')
plt.ylabel('Регистрации')
plt.tight_layout()
plt.savefig('./charts/total_registrations.png')
plt.close()

# Итоговые регистрации по платформам
plt.figure(figsize=(10, 5))
sns.lineplot(data=merged_df, x='date_group', y='registrations', hue='platform')
plt.title('Регистрации по платформам')
plt.xlabel('Дата')
plt.ylabel('Регистрации')
plt.tight_layout()
plt.savefig('./charts/registrations_by_platform.png')
plt.close()

# Конверсия по платформам
plt.figure(figsize=(10, 5))
sns.lineplot(data=merged_df, x='date_group', y='conversion', hue='platform')
plt.title('Конверсия по платформам')
plt.xlabel('Дата')
plt.ylabel('Конверсия (%)')
plt.tight_layout()
plt.savefig('./charts/conversion_by_platform.png')
plt.close()

# Средняя конверсия
plt.figure(figsize=(10, 5))
merged_df.groupby('date_group')['conversion'].mean().plot()
plt.title('Средняя конверсия')
plt.xlabel('Дата')
plt.ylabel('Конверсия (%)')
plt.tight_layout()
plt.savefig('./charts/average_conversion.png')
plt.close()

# Стоимости рекламы
plt.figure(figsize=(10, 5))
ads_merged.set_index('date_group')['cost'].plot()
plt.title('Затраты на рекламу')
plt.xlabel('Дата')
plt.ylabel('Стоимость')
plt.tight_layout()
plt.savefig('./charts/ads_costs.png')
plt.close()

# Визиты с выделением рекламной кампании
plt.figure(figsize=(10, 5))
sns.scatterplot(data=ads_merged, x='date_group', y='visits', hue='utm_campaign')
plt.title('Визиты с рекламой')
plt.tight_layout()
plt.savefig('./charts/visits_with_ads.png')
plt.close()

# Регистрации с выделением рекламной кампании
plt.figure(figsize=(10, 5))
sns.scatterplot(data=ads_merged, x='date_group', y='registrations', hue='utm_campaign')
plt.title('Регистрации с рекламой')
plt.tight_layout()
plt.savefig('./charts/registrations_with_ads.png')
plt.close()


In [2]:
import os
import requests
import pandas as pd
import matplotlib.pyplot as plt
from dotenv import load_dotenv

# Загружаем переменные окружения
load_dotenv()
API_URL = os.getenv('API_URL')
DATE_BEGIN = os.getenv('DATE_BEGIN')
DATE_END = os.getenv('DATE_END')

# Создание папки charts, если нет
os.makedirs('./charts', exist_ok=True)

# Загружаем данные по API
response = requests.get(f"{API_URL}/registrations", params={'begin': DATE_BEGIN, 'end': DATE_END})
registrations = pd.DataFrame(response.json())

# Сохраняем в JSON
registrations.to_json('./registrations.json', orient='records')

# Подготовка данных
registrations['datetime'] = pd.to_datetime(registrations['datetime'])
registrations['date_group'] = registrations['datetime'].dt.date

# Группировка по типу регистрации
grouped = registrations.groupby(['date_group', 'registration_type']).size().unstack(fill_value=0)

# Построение графика
plt.figure(figsize=(14, 7))
for column in grouped.columns:
    plt.plot(grouped.index, grouped[column], label=column)

plt.title('Регистрации по типу регистрации')
plt.xlabel('Дата')
plt.ylabel('Количество регистраций')
plt.legend(title='Тип регистрации')
plt.grid(True)
plt.tight_layout()
plt.savefig('./charts/registrations_by_type.png')
plt.close()
